In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport sklearn.decomposition as decomposition
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 
@pyimport sklearn.cluster as cluster

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
data_info_dict = load("/home/chuyu/Notebooks/project_place_cell/figures/chuyu/figure_data_info.jld2")

data_info_all = []
for key in keys(data_info_dict)
    if key == "corner_cue_circle"
    append!(data_info_all, data_info_dict[key])
    end
end

In [ ]:
data_info_all

In [ ]:
analyzer = "chuyu"

In [ ]:
which_data = 1
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]

    experiment_filename_2 = data_info[3]
    server_2 = data_info[4]

    experimenter = data_info[5]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

    ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
    ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
    ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")


    file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
    all_files = readdir(file_folder_1)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    n_bins = save_file_name[end-4:end-3]
    info_filename = joinpath(file_folder_1, save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    valid_neuron_1 = HDF5.readmmap(file["valid_neurons"])

    close(file)


    file_folder_2 = joinpath(data_path(ds_save_cy_2), "")
    all_files = readdir(file_folder_2)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_2, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    println(candidate_filename[which_file])
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(file_folder_2, save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_2 = HDF5.readmmap(file["place_map_all"])
    valid_neuron_2 = HDF5.readmmap(file["valid_neurons"])

    close(file)


    NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
    NMF_file = h5open(NMF_filename, "r")
    global Z_all = HDF5.readmmap(NMF_file["Z_all"])
    global X_all = HDF5.readmmap(NMF_file["X_all"])
    global Y_all = HDF5.readmmap(NMF_file["Y_all"])
    global A_dFF_1 = HDF5.readmmap(NMF_file["A_dF"]); # bs update
    close(NMF_file)

    # for multi sessions
    NMF_filename = joinpath(data_path(ds_save_cy_2), "NMF_merge.h5") # bs update
    NMF_file = h5open(NMF_filename, "r") # bs update
    global A_dFF_2 = HDF5.readmmap(NMF_file["A_dF"]); # bs update
    close(NMF_file) # bs update

    n_neuron = length(X_all);

    save_file_name = "compare_map_results.h5"
    compare_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
    file = h5open(compare_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    confined_place_cell_index_1 = HDF5.readmmap(file["confined_place_cell_index_1"])
    confined_place_cell_index_2 = HDF5.readmmap(file["confined_place_cell_index_2"])
    peak_loc_map_1 = HDF5.readmmap(file["peak_loc_map_1"])
    peak_loc_map_2 = HDF5.readmmap(file["peak_loc_map_2"])
    close(file)




In [ ]:
tel_place_cell_index = intersect(tel_place_cell_index, valid_neuron_1, valid_neuron_2)

In [ ]:
    file_folder_1 = joinpath(data_path(ds_save_cy_1), "place_cell_windows")
    all_files = readdir(file_folder_1)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    save_file_name = candidate_filename[which_file][1]
    println(save_file_name)   
    info_filename = joinpath(data_path(ds_save_cy_1),"place_cell_windows", save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1_early = HDF5.readmmap(file["place_map_all"])
    close(file)
        
    save_file_name = candidate_filename[which_file][2]
    println(save_file_name)
    info_filename = joinpath(data_path(ds_save_cy_1),"place_cell_windows", save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1_late = HDF5.readmmap(file["place_map_all"])
    specificity_1_late = HDF5.readmmap(file["specificity"])
    close(file)

    which_neuron = tel_place_cell_index

    corr_map_pairwise_all_1_early = fill(NaN32, length(which_neuron), length(which_neuron))

    for (i, neuron_idx_1) in enumerate(which_neuron)
        for (j, neuron_idx_2) in enumerate(which_neuron)
            map_1 = place_map_all_1_early[:,:, neuron_idx_1]
            map_2 = place_map_all_1_early[:,:, neuron_idx_2]
            corr_map_pairwise_all_1_early[i,j] = corr_2d(map_1, map_2)
        end
    end

    corr_map_pairwise_all_1_late = fill(NaN32, length(which_neuron), length(which_neuron))

    for (i, neuron_idx_1) in enumerate(which_neuron)
        for (j, neuron_idx_2) in enumerate(which_neuron)
            map_1 = place_map_all_1_late[:,:, neuron_idx_1]
            map_2 = place_map_all_1_late[:,:, neuron_idx_2]
            corr_map_pairwise_all_1_late[i,j] = corr_2d(map_1, map_2)
        end
    end





    # histogram of PF correlation
    which_neuron = tel_place_cell_index

    corr_map_pairwise_all_1 = fill(NaN32, length(which_neuron), length(which_neuron))

    for (i, neuron_idx_1) in enumerate(which_neuron)
        for (j, neuron_idx_2) in enumerate(which_neuron)
            map_1 = place_map_all_1[:,:, neuron_idx_1]
            map_2 = place_map_all_1[:,:, neuron_idx_2]
            corr_map_pairwise_all_1[i,j] = corr_2d(map_1, map_2)
        end
    end

    corr_map_pairwise_all_2 = fill(NaN32, length(which_neuron), length(which_neuron))

    for (i, neuron_idx_1) in enumerate(which_neuron)
        for (j, neuron_idx_2) in enumerate(which_neuron)
            map_1 = place_map_all_2[:,:, neuron_idx_1]
            map_2 = place_map_all_2[:,:, neuron_idx_2]
            corr_map_pairwise_all_2[i,j] = corr_2d(map_1, map_2)
        end
    end



    which_neuron = tel_place_cell_index
    cell_locs = reduce(hcat, [X_all, Y_all, Z_all.*2])[which_neuron,:];
    anatomy_pairwise_distance =  metrics.pairwise_distances(cell_locs);



In [ ]:
@pyimport sklearn.metrics as metrics

In [ ]:
@showprogress for which_data = 1:length(data_info_all)
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]

    experiment_filename_2 = data_info[3]
    server_2 = data_info[4]

    experimenter = data_info[5]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

    ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
    ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
    ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")


    file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
    all_files = readdir(file_folder_1)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    n_bins = save_file_name[end-4:end-3]
    info_filename = joinpath(file_folder_1, save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    close(file)


    file_folder_2 = joinpath(data_path(ds_save_cy_2), "")
    all_files = readdir(file_folder_2)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_2, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    println(candidate_filename[which_file])
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(file_folder_2, save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_2 = HDF5.readmmap(file["place_map_all"])
    close(file)


    NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
    NMF_file = h5open(NMF_filename, "r")
    global Z_all = HDF5.readmmap(NMF_file["Z_all"])
    global X_all = HDF5.readmmap(NMF_file["X_all"])
    global Y_all = HDF5.readmmap(NMF_file["Y_all"])
    global A_dFF_1 = HDF5.readmmap(NMF_file["A_dF"]); # bs update
    close(NMF_file)

    # for multi sessions
    NMF_filename = joinpath(data_path(ds_save_cy_2), "NMF_merge.h5") # bs update
    NMF_file = h5open(NMF_filename, "r") # bs update
    global A_dFF_2 = HDF5.readmmap(NMF_file["A_dF"]); # bs update
    close(NMF_file) # bs update

    n_neuron = length(X_all);

    save_file_name = "compare_map_results.h5"
    compare_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
    file = h5open(compare_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
    confined_place_cell_index_1 = HDF5.readmmap(file["confined_place_cell_index_1"])
    confined_place_cell_index_2 = HDF5.readmmap(file["confined_place_cell_index_2"])
    peak_loc_map_1 = HDF5.readmmap(file["peak_loc_map_1"])
    peak_loc_map_2 = HDF5.readmmap(file["peak_loc_map_2"])
    close(file)

    file_folder_1 = joinpath(data_path(ds_save_cy_1), "place_cell_windows")
    all_files = readdir(file_folder_1)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    save_file_name = candidate_filename[which_file][1]
    println(save_file_name)   
    info_filename = joinpath(data_path(ds_save_cy_1),"place_cell_windows", save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1_early = HDF5.readmmap(file["place_map_all"])
    close(file)
        
    save_file_name = candidate_filename[which_file][2]
    println(save_file_name)
    info_filename = joinpath(data_path(ds_save_cy_1),"place_cell_windows", save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1_late = HDF5.readmmap(file["place_map_all"])
    specificity_1_late = HDF5.readmmap(file["specificity"])
    close(file)

    which_neuron = tel_place_cell_index

    corr_map_pairwise_all_1_early = fill(NaN32, length(which_neuron), length(which_neuron))

    for (i, neuron_idx_1) in enumerate(which_neuron)
        for (j, neuron_idx_2) in enumerate(which_neuron)
            map_1 = place_map_all_1_early[:,:, neuron_idx_1]
            map_2 = place_map_all_1_early[:,:, neuron_idx_2]
            corr_map_pairwise_all_1_early[i,j] = corr_2d(map_1, map_2)
        end
    end

    corr_map_pairwise_all_1_late = fill(NaN32, length(which_neuron), length(which_neuron))

    for (i, neuron_idx_1) in enumerate(which_neuron)
        for (j, neuron_idx_2) in enumerate(which_neuron)
            map_1 = place_map_all_1_late[:,:, neuron_idx_1]
            map_2 = place_map_all_1_late[:,:, neuron_idx_2]
            corr_map_pairwise_all_1_late[i,j] = corr_2d(map_1, map_2)
        end
    end



    pairwise_distance_peak_1 = metrics.pairwise_distances(peak_loc_map_1[:,whether_in(confined_place_cell_index_1, confined_place_cell_index_2)]')
    pairwise_distance_peak_2 = metrics.pairwise_distances(peak_loc_map_2[:,whether_in(confined_place_cell_index_2, confined_place_cell_index_1)]');


    # histogram of PF correlation
    which_neuron = tel_place_cell_index

    corr_map_pairwise_all_1 = fill(NaN32, length(which_neuron), length(which_neuron))

    for (i, neuron_idx_1) in enumerate(which_neuron)
        for (j, neuron_idx_2) in enumerate(which_neuron)
            map_1 = place_map_all_1[:,:, neuron_idx_1]
            map_2 = place_map_all_1[:,:, neuron_idx_2]
            corr_map_pairwise_all_1[i,j] = corr_2d(map_1, map_2)
        end
    end

    corr_map_pairwise_all_2 = fill(NaN32, length(which_neuron), length(which_neuron))

    for (i, neuron_idx_1) in enumerate(which_neuron)
        for (j, neuron_idx_2) in enumerate(which_neuron)
            map_1 = place_map_all_2[:,:, neuron_idx_1]
            map_2 = place_map_all_2[:,:, neuron_idx_2]
            corr_map_pairwise_all_2[i,j] = corr_2d(map_1, map_2)
        end
    end




    # histogram of PF correlation
    which_neuron = tel_place_cell_index

    corr_activity_pairwise_all_1 = fill(NaN32, length(which_neuron), length(which_neuron))

    for (i, neuron_idx_1) in enumerate(which_neuron)
        for (j, neuron_idx_2) in enumerate(which_neuron)
            activity_1 = A_dFF_1[:, neuron_idx_1]
            activity_2 = A_dFF_1[:, neuron_idx_2]
            corr_activity_pairwise_all_1[i,j] = corr_nan(activity_1, activity_2)
        end
    end

    corr_activity_pairwise_all_2 = fill(NaN32, length(which_neuron), length(which_neuron))

    for (i, neuron_idx_1) in enumerate(which_neuron)
        for (j, neuron_idx_2) in enumerate(which_neuron)
            activity_1 = A_dFF_2[:, neuron_idx_1]
            activity_2 = A_dFF_2[:, neuron_idx_2]
            corr_activity_pairwise_all_2[i,j] = corr_nan(activity_1, activity_2)
        end
    end

    which_neuron = tel_place_cell_index
    cell_locs = reduce(hcat, [X_all, Y_all, Z_all.*2])[which_neuron,:];
    anatomy_pairwise_distance =  metrics.pairwise_distances(cell_locs);


    h5open(joinpath(data_path(ds_save_analyzer_1), "cluster_analysis.h5"), "w") do file
        file["corr_map_pairwise_all_1_early"] = corr_map_pairwise_all_1_early
        file["corr_map_pairwise_all_1_late"] = corr_map_pairwise_all_1_late
        file["pairwise_distance_peak_1"] = pairwise_distance_peak_1
        file["pairwise_distance_peak_2"] = pairwise_distance_peak_2
        file["corr_map_pairwise_all_1"] = corr_map_pairwise_all_1
        file["corr_map_pairwise_all_2"] = corr_map_pairwise_all_2
        file["corr_activity_pairwise_all_1"] = corr_activity_pairwise_all_1;
        file["corr_activity_pairwise_all_2"] = corr_activity_pairwise_all_2;
        file["anatomy_pairwise_distance"] = anatomy_pairwise_distance;   

    end;


end

In [ ]:
for which_data in 1:length(data_info_all)
    try
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]

    experiment_filename_2 = data_info[3]
    server_2 = data_info[4]

    experimenter = data_info[5]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

    ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
    ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
    ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")


    all_files = readdir(data_path(ds_save_cy_1))
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="compar" for i in 1:length(long_name_files)])
    # println(long_name_files[spatial_info_index])
    
    
    # println(which_data)
    
    path_target = data_path(ds_save_cy_1)
    run(`ls -lha $path_target/cluster_analysis.h5`)

    catch e
        println(e)
    end
    

end